### Graded Lab 4

Hello ! Welcome to Graded Lab of Module 4.

In the last assignment we had worked on correlation , hypothesis which business wanted us to test.
Its high time when we look at the missing values & outliers stuff.
**Here we will work on missing values & outliers.**

In case if you are not able to recollect the problem description and data description then mentioning it below.

Lets look at the problem statement,

*Client: ABC Retail, Incorporated, rest-of-the-world division* 

***Project name: Online retail sales analysis*** 

An online retailer, ABC, Inc., operates in nearly 100 countries worldwide, selling furniture, office supplies and technology products to customers in three segments: consumer, corporate and home office. ABC, Inc. is a US-based company, and it has two major divisions: US and rest of the world. We are working with the rest of the world division of the company. 

They have provided us with online sales transaction data from 2011 to 2014.

We are given 3 datasets:-

1. Data on each sale; 51290 records; all data in US dollars
It contains fields like
**order_id** (identifier) ,order_date ,ship_date ,ship_mode ,**customer_id**(identifier) ,product_id ,category ,sub_category ,product_name ,sales ,quantity ,discount ,profit ,shipping_cost ,order_priority ,**vendor_code** (identifier) 


2. Data on the customers; 1590 records 
It contains fields like
**customer_id** (identifier) ,customer_name ,city ,state ,country ,postal_code ,segment ,market ,region 

3. Data on vendors who supply the retailer; 65 records 
It contains fields like
vendor ,**vendor_code** (identifier) 

We need to analyze the data and need to provide answer to different questions asked by company officials.

# Instructor update: Jan 29, 2024

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from itertools import combinations
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

### Reading sales data
sales = pd.read_csv('sales_data_M4.csv')

### Reading customer data
cust = pd.read_csv(r'customers.csv',encoding='iso-8859-1')

### Reading vendor data
vend = pd.read_csv(r'vendors.csv')

sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code
0,1,MX-2014-143658,02/10/2014,06/10/2014,Standard Class,SC-20575,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,4.56,1.03,Medium,VE_001
1,2,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,90.72,13.45,Medium,VE_002
2,3,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,54.08,9.63,Medium,VE_003
3,4,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,4.96,1.37,Medium,VE_004
4,5,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,11.44,3.79,Medium,VE_005


In order to solve the next questions , we need to combine all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. So here we have written a data processing function.
There are 2 tasks which are to be performed.
1. Merge/ Join all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. (Understand which should be the joining key , type of join , refer .merge() function of pandas)
2. Convert 'order_date' into a datetime column.
**Return output as a dataframe**

In [2]:
#### data_merging & order_date processing , data1 will be sales , data2 will be customer dataset & data3 will be vendor dataset.

def data_process(data1,data2,data3):
    
    # Merge data1 and data2 on 'customer_id'

    data = pd.merge(data1, data2, on='customer_id', how='left')

    # Merge the result with data3 on 'vendor_code'

    data = pd.merge(data, data3, on='vendor_code', how='left')

    # Convert 'order_date' into a datetime column

    data['order_date'] = pd.to_datetime(data['order_date'])

    return data

In [3]:
sales= data_process(data1=sales.copy(),data2=cust.copy(),data3=vend.copy())

In [4]:
assert sales['order_date'].dtypes=='<M8[ns]' ,'Make sure if you have converted order_date into a datetime format correctly or not.'
assert sales.shape== (51290,26) ,'Checking size and shape of dataframe after merging is a very important check.'

In [5]:
sales['order_date'].dtypes

dtype('<M8[ns]')

In [6]:
sales.shape

(51290, 26)

In [7]:
#sales.to_csv("salesmissing.csv", index=False)

Note that in this dataset we are purposefully introducing some of the missing values.

### Q.1 Return column names & missing values percentage for the columns which has missing value% >50. Output should be a dictionary. 

For eg:- { column_name : value%} i.e {'A':54.50,'B':59.75}, make sure to round off percentages to 2 decimals.

In [8]:
def missing_value_col(data):
    
    # Calculate the percentage of missing values for each column
    missing_percentages = data.isnull().mean() * 100

    # Filter out columns which have more than 50% missing values
    missing_percentages = {col: round(percent, 2) for col, percent in missing_percentages.items() if percent > 50}

    return missing_percentages

In [9]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

In [10]:
missing_value_col(data=sales) ### PASSED

{'postal_code': 100.0}

#### **Senior data scientists after consulting with the business have decided that such columns needs to be dropped before going further.**

In [11]:
high_missing_col=list(missing_value_col(data=sales).keys()) #Enter the columns with high % of missing values
### We will drop columns with such high missing values
sales.drop(high_missing_col,axis=1,inplace=True)

In [12]:
sales.shape

(51290, 25)

### Q.2 Fill the missing data in 'category' column with appropriate method.  
(Hint:- Remember variable 'category' & 'sub-category' are related & hierachy goes like this category -> sub-category)

Approach:-

1. Create a key-pair dictionary where key will be product category & pair values will be sub-categories.

2. Wherever category value is missing check for corresponding sub-category value. 

3. From sub-category value trace back its original product category value by looking in dictionary, get a product value.

4. Fill the blank product category value by the same.

In [13]:
### create a dictionary where keys will be categories & sub-categories as values. store in inside Category_dict.

def cat_dict(data,main_col,sub_col):
    
    # Create a dictionary where keys are categories and values are lists of sub-categories
    Category_dict = data.groupby(main_col)[sub_col].apply(list).to_dict()
    # Remove duplicates in each list
    Category_dict = {k: list(set(v)) for k, v in Category_dict.items()}
    return Category_dict 

def category_subcategory_map_dataframe(df, Category_dict, main_col, sub_col):
    def category_subcategory_map(row):
        if pd.isna(row[main_col]):  # If the main column is null
            if not pd.isna(row[sub_col]):  # If the sub column is not null
                for k, v in Category_dict.items():
                    if row[sub_col] in Category_dict[k]:
                        return k  # k is the correct sub-category for the category value v
        else:
            return row[main_col]  # If the main column is not null, return its existing value

    df[main_col] = df.apply(category_subcategory_map, axis=1)
    return df[main_col]

In [14]:
Category_dict=cat_dict(data=sales,main_col='category',sub_col='sub_category')
sales['category']=category_subcategory_map_dataframe(df=sales, Category_dict=Category_dict, main_col='category', sub_col='sub_category')

In [15]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

In [16]:
sales['category']  ### PASSED

0        Office Supplies
1              Furniture
2              Furniture
3        Office Supplies
4        Office Supplies
              ...       
51285    Office Supplies
51286         Technology
51287    Office Supplies
51288    Office Supplies
51289    Office Supplies
Name: category, Length: 51290, dtype: object

### Q.3 ) Fill the missing values in 'sales' column by average value of that column. Return output series.

In [17]:
def fill_missing(data,col):
    
    # Calculate the mean of the column
    mean_val = data[col].mean()
    
    # Fill missing values with the mean
    data[col] = data[col].fillna(mean_val)

    return data[col]

In [18]:
sales['sales']=fill_missing(data=sales,col='sales')

In [19]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert sales['sales'].isna().sum()==0 ,'Missing values found for sales.'

In [20]:
sales['sales'].isna().sum() ### PASSED

0

### Q.4 ) Discretize the 'sales' column wrto following groups. Each group should contain number of orders as values.
Groups :- [0,12),[12,37),[37,113),[113,186),[186,1200).

**Note :-[a,b) indicates that group includes a but excudes b.**

**Return output series with group name as index & number of orders as values.**

For eg:-

[186, 1200)             123

[37, 113)               123

[12, 37)                123

[113, 186)              123

[0, 12)                 123

Name: sales, dtype: int64

In [21]:
def manual_bin(data,col):
    
    # Define the bin edges
    bins = [0, 12, 37, 113, 186, 1200]
    
    # Use pandas cut function to bin the data
    bins_result = pd.cut(data[col], bins, right=False)
    
    # Use value_counts to count the number of orders in each bin, then sort by values in descending order
    result_series = bins_result.value_counts().sort_values(ascending=False)
    
    return result_series

In [22]:
assert type(manual_bin(data=sales,col='sales'))==pd.Series, "Please provide output in series format."

In [23]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert manual_bin(data=sales,col='sales').values[0]==18550, 'Make sure you have returned correct series values.'

In [24]:
manual_bin(data=sales,col='sales')

[186, 1200)    18550
[37, 113)      12761
[12, 37)        9325
[113, 186)      5187
[0, 12)         3708
Name: sales, dtype: int64

In [25]:
type(manual_bin(data=sales,col='sales'))

pandas.core.series.Series

In [26]:
manual_bin(data=sales,col='sales').values[0]  ### PASSED

18550

### Q.5 )  Sometimes, it is difficult to comprehend the quantity sold ('quantity') value as a numeric value. So, the business team wants to observe the sales as High, Medium and Low categories.  

### Write a code to discretize the sales data and  return an output series containing High, Medium and Low categories as index & the respective number of records as values.

Hint : Here the bounds for High , Medium , Low categories are not given hence we can consider the equal split of data for defining the bounds , think of using pd.cut() function.

In [27]:
# def discrete_bin(data,col):
    
#     # Define the bins for High, Medium, and Low
#     bins = [float('-inf'), data[col].quantile(1/3), data[col].quantile(2/3), float('inf')]
    
#     # Define the labels for the bins
#     labels = ['Low', 'Medium', 'High']
    
#     # Cut the 'quantity' column into bins
#     bins_result = pd.cut(data[col], bins=bins, labels=labels, include_lowest=True, right=False)
    
#     # Count the number of records in each category
#     result_series = bins_result.value_counts().sort_index(ascending=False)
    
#     return result_series

In [28]:
def discrete_bin(data, col):
    
    bins = pd.cut(data[col], bins=3, labels=['Low', 'Medium', 'High'])
    return bins.value_counts().sort_index(ascending=False)

In [29]:
assert type(discrete_bin(data=sales,col='quantity'))==pd.Series, "Please provide output in series format."

In [30]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

In [31]:
discrete_bin(data=sales,col='quantity')

High        877
Medium     7753
Low       42660
Name: quantity, dtype: int64

In [32]:
type(discrete_bin(data=sales,col='quantity')) ### PASSED

pandas.core.series.Series

### Q.6. In order to perform further analysis, we intend to utilize the 'ship_mode' variable, which has categorical values ('First Class'(0) > 'Second Class'(1) > 'Standard Class'(2) > 'Same Day'(3)). To facilitate this analysis, we need to convert 'ship_mode' into a numerical feature. Please encode 'ship_mode' into a numerical format.
### Return output series.

Hint : Think of using the OrdinalEncoder() function from scikit-learn.

In [33]:
## Select Appropriate priority

priority = {'First Class': 0, 'Second Class': 1, 'Standard Class': 2, 'Same Day': 3}

def transforming_ordered_var(data,col,priority,nan_val):
    #data: dataset to be used
    #col : column name which is to be encoded.
    #priority: list of categories sorted from highest order of importance to lowest order importance.
    #nan_val : it will be a value assigned in place of nan's.   
    data[col] = data[col].map(priority).fillna(nan_val).astype(int)

    return data[col]

In [34]:
sales['shipping_mode_encoded']=transforming_ordered_var(data=sales,col='ship_mode',priority=priority,nan_val=5)

In [35]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert int(sales['shipping_mode_encoded'].value_counts().loc[lambda x:x.index==5])==1570

In [36]:
sales['shipping_mode_encoded']

0        2
1        2
2        2
3        2
4        2
        ..
51285    0
51286    5
51287    0
51288    0
51289    2
Name: shipping_mode_encoded, Length: 51290, dtype: int64

In [37]:
sales['shipping_mode_encoded'].value_counts()

2    29842
1     9985
0     7272
3     2621
5     1570
Name: shipping_mode_encoded, dtype: int64

In [38]:
sales['shipping_mode_encoded'].value_counts().loc[lambda x:x.index==5]

5    1570
Name: shipping_mode_encoded, dtype: int64

### Q.7) Outliers can be defined in different ways. For each of the following definitions of outliers, what percent of the values in the  'sales' are considered outliers? Give the answer as a percent rounded upto two decimal places.


Note -
1. 'IQR_detection' : Less than (Q1 - 1.5*IQR) and greater than (Q3 + 1.5*IQR) are considered outliers

2. 'percentile_detection': Below the 3rd percentile and above the 97th percentile are considered outliers

3. 'mean_SD' : Less than (mean - 3*SD) and greater than (mean + 3*SD) are considered outliers 

In [39]:
def detect_outliers(data,col,method):
    
    if method=='IQR_detection':
        # Calculate the first quartile (Q1) and the third quartile (Q3)
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        
        # Calculate the interquartile range (IQR)
        IQR = Q3 - Q1
        
        # Determine the lower and upper bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count the number of outliers
        outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
        
    elif method=='percentile_detection':
        
        # Calculate the 3rd and 97th percentiles
        percentile_3 = data[col].quantile(0.03)
        percentile_97 = data[col].quantile(0.97)
        
        # Determine the lower and upper bounds for outliers
        lower_bound = percentile_3
        upper_bound = percentile_97
        
        # Count the number of outliers
        outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
        
    elif method=='mean_SD':
        # Calculate the mean and standard deviation (SD)
        mean = data[col].mean()
        std_dev = data[col].std()
        
        # Determine the lower and upper bounds for outliers
        lower_bound = mean - 3 * std_dev
        upper_bound = mean + 3 * std_dev
        
        # Count the number of outliers
        outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
        
    outliers_perc = float((outliers.sum() / len(data[col])) * 100)
    
    outliers_perc = round(outliers_perc,2)
            
    return outliers_perc

In [40]:
assert type(detect_outliers(data=sales,col='sales',method='IQR_detection'))==np.float, 'Make sure that output is returned as float'
assert detect_outliers(data=sales,col='sales',method='IQR_detection')==10.06, 'Make sure that output is rounded upto 2 decimals'

In [41]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

In [42]:
detect_outliers( data=sales, col='sales', method='IQR_detection')

10.06

In [43]:
type(detect_outliers(data=sales,col='sales',method='IQR_detection')) ### PASSED

float

### Q.8) Detect upper end outliers in profit column by 'IQR_detection'. Return number of upper end outliers.

In [44]:
def detect_upperend_outliers(data,col):
    
    # Calculate the first quartile (Q1) and the third quartile (Q3)
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
        
    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1
        
    # Determine the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
        
    # Count the number of outliers
    upper_outliers = int((data[col] > upper_bound).sum())
        
    return upper_outliers

In [45]:
assert type(detect_upperend_outliers(data=sales,col='profit'))==int , 'No of outliers should be an integer , make sure that you are returning output in integer format.'

In [46]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

In [47]:
detect_upperend_outliers(data=sales,col='profit')

6229

In [48]:
type(detect_upperend_outliers(data=sales,col='profit'))  ### PASSED

int

### Q.9) Cap & Floor outliers at 97% at 3%. Return the processed column as output.

In [49]:
def cap_and_floor_column(column, cap_percentile, floor_percentile):
    
    # Calculate the 3rd and 97th percentiles
    percentile_3 = column.quantile(0.03)
    percentile_97 = column.quantile(0.97)
        
    # Determine the lower and upper bounds for outliers
    lower_bound = percentile_3
    upper_bound = percentile_97
        
    # Cap and floor the column based on the bounds
    capped_and_floored_column = column.clip(lower=lower_bound, upper=upper_bound)
    
    return capped_and_floored_column ## Capped ,Floored series

In [50]:
sales['profit_outlier_capped'] = cap_and_floor_column(column=sales['profit'], cap_percentile=97, floor_percentile=3)

In [51]:
assert round(np.quantile(sales['profit_outlier_capped'],0.97),4)==301.9693, 'Make sure that you are flooring and capping with correct percentiles & returning the correct output.'
assert round(np.quantile(sales['profit_outlier_capped'],0.03),4)==-144.632, 'Make sure that you are flooring and capping with correct percentiles & returning the correct output.'

In [52]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

In [53]:
np.quantile(sales['profit_outlier_capped'],0.97)

301.9693068

In [54]:
np.quantile(sales['profit_outlier_capped'],0.03)  ### PASSED

-144.6319602

### Q.10) Business team is interested in knowing how many vendors have profit_outlier_capped <10000 & vendors with profit_outlier_capped >10000. Return the dataframe with 3 columns vendor  , profit_outlier_capped , profit_bins & sort it with 'profit_outlier_capped' in descending order.

Instructions:-
1. Bins should be in (a,b] format. (a,b] indicates that bin excludes a but incudes b.
2. Make sure that for **1st interval only** it should be **left inclusive** i.e [a,b].
3. Segment the ''profit_outlier_capped'' variable in 2 bins. (binendpoints should be min value of profit<10000,10000,max value of profit>10000).

In [55]:
def profit_vendor(data,col):
    
    # Group by vendor and calculate the sum of profit_outlier_capped
    salesgrp = data.groupby("vendor", as_index=False)[col].sum()

    # Define bin cut-points
    bin_cut_points = [salesgrp[col].min(), 10000, salesgrp[col].max()]

    # Create profit_bins column using cut
    salesgrp['profit_bins'] = pd.cut(salesgrp[col], bins=bin_cut_points, include_lowest=True, right=True)

    # Sort the DataFrame by 'profit_outlier_capped' in descending order
    salesgrp = salesgrp.sort_values(by=col, ascending=False)

    return salesgrp[['vendor', col, 'profit_bins']]

In [56]:
assert type(profit_vendor(data=sales,col='profit_outlier_capped'))==pd.DataFrame ,'Make sure that your output is a dataframe '
assert str(profit_vendor(data=sales,col='profit_outlier_capped')['profit_bins'].unique()[0])=='(10000.0, 76469.853]' , 'Make sure that you are selecting bin cut-points(a & b) properly'
assert str(profit_vendor(data=sales,col='profit_outlier_capped')['profit_bins'].unique()[1])=='(2732.9049999999997, 10000.0]', 'Make sure that you are selecting bin cut-points(a & b) properly'

In [57]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

In [58]:
profit_vendor(data=sales,col='profit_outlier_capped')

,vendor,profit_outlier_capped,profit_bins
44,Proton Solutions,76469.85264,"(10000.0, 76469.853]"
28,Low Tide Corp,51929.66308,"(10000.0, 76469.853]"
59,Venusystems,49741.37212,"(10000.0, 76469.853]"
53,Sharkfin Networks,47919.12612,"(10000.0, 76469.853]"
26,Lemonetworks,45814.85148,"(10000.0, 76469.853]"
...,...,...,...
31,Moondustries,5287.96536,"(2732.9049999999997, 10000.0]"
29,Luckydream,5243.19210,"(2732.9049999999997, 10000.0]"
43,Prodintelligence,4747.80126,"(2732.9049999999997, 10000.0]"
5,Apexman,4583.38326,"(2732.9049999999997, 10000.0]"


In [59]:
(profit_vendor(data=sales,col='profit_outlier_capped')['profit_bins'].unique()[0])

Interval(10000.0, 76469.853, closed='right')

In [60]:
(profit_vendor(data=sales,col='profit_outlier_capped')['profit_bins'].unique()[1])

Interval(2732.9049999999997, 10000.0, closed='right')

# 80/100 points earned